# QUICKSTART
#### This section runs through the API for common tasks in machine learning. Refer to the links in each section to dive deeper.

## Working with data
#### PyTorch has two primitives to work with data: torch.utils.data.DataLoader and torch.utils.data.Dataset. Dataset stores the samples and their corresponding labels, and DataLoader wraps an iterable around the Dataset.

In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

## Importing Datasets
#### The torchvision.datasets module contains Dataset objects for many real-world vision data like CIFAR, COCO (full list here). In this tutorial, we use the FashionMNIST dataset. Every TorchVision Dataset includes two arguments: transform and target_transform to modify the samples and labels respectively.



In [2]:
# Download training dataset
training_data = datasets.FashionMNIST(
    root='data',
    train = True,
    download=True,
    transform = ToTensor()
)
#  Download test dataset
test_data = datasets.FashionMNIST(
    root = 'data',
    train = False,
    download = True,
    transform = ToTensor()
)

#### We pass the Dataset as an argument to DataLoader. This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element in the dataloader iterable will return a batch of 64 features and labels.

In [4]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
for X,y in test_dataloader:
    print(X.shape, y.shape)
    break
print(len(train_dataloader),len(test_dataloader))

torch.Size([64, 1, 28, 28]) torch.Size([64])
938 157


## Creating Models
#### To define a neural network in PyTorch, we create a class that inherits from nn.Module. We define the layers of the network in the __init__ function and specify how data will pass through the network in the forward function. To accelerate operations in the neural network, we move it to the GPU if available.

In [57]:
# Get cpu or gpu device for training.
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {device} device')

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork,self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512,512),
            nn.ReLU(),
            nn.Linear(512 ,50)
        )
    def forward(self , x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
model = NeuralNetwork().to(device)
print(model)


Using {device} device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=50, bias=True)
  )
)


## Optimizing the Model Parameters
#### To train a model, we need a loss function and an optimizer.



In [58]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr= 1e-3)

## Training Function
#### In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.

In [59]:
def train(data_loader, model, loss_fn, optimizer):
    size = len(data_loader)
    model.train()
    for batch, (X, y) in enumerate(data_loader):
        X, y = X.to(device), y.to(device)

        pred = model(X) 
        loss = loss_fn(pred,y) #calculate loss between predicted and actual labels

        optimizer.zero_grad() # initialize gradients to zeros
        loss.backward() # backpropagate loss
        optimizer.step() # Update the parameters

        if batch % 1000 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

## Test Function
#### We also check the model’s performance against the test dataset to ensure it is learning.

In [60]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    # print(f"Size {size} and batches {num_batches} =")
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
            # print(pred.argmax(0),'--', y)
    # print(f"test_loss {test_loss} and correct {correct}")
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

#### The training process is conducted over several iterations (epochs). During each epoch, the model learns parameters to make better predictions. We print the model’s accuracy and loss at each epoch; we’d like to see the accuracy increase and the loss decrease with every epoch.

In [61]:
epochs = 20
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 3.914723  [    0/  938]
Test Error: 
 Accuracy: 16.7%, Avg loss: 2.467575 

Epoch 2
-------------------------------
loss: 2.527528  [    0/  938]
Test Error: 
 Accuracy: 55.4%, Avg loss: 1.824504 

Epoch 3
-------------------------------
loss: 1.907827  [    0/  938]
Test Error: 
 Accuracy: 60.4%, Avg loss: 1.420708 

Epoch 4
-------------------------------
loss: 1.520391  [    0/  938]
Test Error: 
 Accuracy: 63.4%, Avg loss: 1.193117 

Epoch 5
-------------------------------
loss: 1.289641  [    0/  938]
Test Error: 
 Accuracy: 65.0%, Avg loss: 1.052354 

Epoch 6
-------------------------------
loss: 1.137487  [    0/  938]
Test Error: 
 Accuracy: 66.4%, Avg loss: 0.958025 

Epoch 7
-------------------------------
loss: 1.026699  [    0/  938]
Test Error: 
 Accuracy: 67.8%, Avg loss: 0.891772 

Epoch 8
-------------------------------
loss: 0.942531  [    0/  938]
Test Error: 
 Accuracy: 68.9%, Avg loss: 0.843025 

Epoch 9
----------------

## Saving Models
#### A common way to save a model is to serialize the internal state dictionary (containing the model parameters).

In [12]:
torch.save(model.state_dict(),'model.pth')
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


## Loading Models
#### The process for loading a model includes re-creating the model structure and loading the state dictionary into it.

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load('model.pth'))

#### This model can now be used to make predictions.

In [69]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[892][0], test_data[888][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(pred[0])
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

tensor([ 2.2116, -3.1041,  3.1737,  4.3450,  4.3037, 15.3547,  3.4812, 15.9296,
        11.5362, 17.5796, -1.2561, -2.3689, -1.4489, -1.5664, -1.8327, -1.3153,
        -2.4469, -1.9401, -1.9762, -2.4275, -2.9579, -2.3499, -1.8568, -1.7041,
        -1.9207, -1.6579, -1.4837, -2.1005, -1.7323, -1.6832, -1.1187, -2.2288,
        -1.4996, -1.6301, -0.7010, -1.9533, -3.2474, -2.3780, -1.8299, -1.0792,
        -1.7036, -2.0266, -1.8191, -2.0812, -1.6671, -1.6344, -1.8994, -1.1810,
        -2.3073, -2.0173])
Predicted: "Ankle boot", Actual: "Coat"
